In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from convert_pmt_ids import *

In [2]:
vm = input()
if vm.lower()=='yes':
    path='/mnt/cephfs/ml_data/mc_2021/'
else:
    path=''

yes


In [3]:
mode = input("Enter mode: nPE/FHT... ", )

Enter mode: nPE/FHT... FHT


In [4]:
if mode=="nPE":
    ind=1
    bins = np.linspace(0, 15, 61)
    bins[-1] = 100
    xaxis_title="nPE"
    y_range = [0, 1100]
    dtick = 1
elif mode=="FHT":
    ind=2
    bins = np.linspace(0, 1000, 51)
    bins[-1] = 1200
    xaxis_title="t, ns"
    y_range = [0, 675]
    dtick = 100

In [5]:
all_data_in_bins = []

In [ ]:
R_lower = 15.995
R_upper = 16.005
energy = 4

for type_of_particle in ['e+', 'gamma']:
    for option in ['ideal', 'real']:
    
        if type_of_particle=='gamma':
            energy+=1

        data = np.load(
            '{0}raw_data/{1}/{2}/test/data/{1}_{3}/raw_data_test_0.npz'.format(path, type_of_particle, option, energy),
            allow_pickle=True)['a']
        data_true = pd.read_csv(
            '{0}raw_data/{1}/{2}/test/targets/{1}_{3}/targets_test_0.csv'.format(path, type_of_particle, option, energy))

        for i in tqdm(range(100)):
            data = np.append(
                data,
                np.load(
                    '{0}raw_data/{1}/{2}/test/data/{1}_{3}/raw_data_test_{4}.npz'.format(
                        path, type_of_particle, option, energy, i), allow_pickle=True)['a'],
                axis=1
            )

            data_true = data_true.append(
                pd.read_csv(
                    '{0}raw_data/{1}/{2}/test/targets/{1}_{3}/targets_test_{4}.csv'.format(
                        path, type_of_particle, option, energy, i))
            )
            data_true['edepR'] = (data_true['edepX']**2 + data_true['edepY']**2 + data_true['edepZ']**2)**0.5

        data_true = data_true.reset_index().drop('index', axis=1)
        data_true['edepR'] = (data_true['edepX']**2 + data_true['edepY']**2 + data_true['edepZ']**2)**0.5

        NEvents = data.shape[1]
        for i in tqdm(range(NEvents)):
            nonzeros_inds = np.where(data[2, :][i] != 0.0)[0]
            data[0, :][i] = convert_pmt_ids(data[0, :][i])
            data[0, :][i] = data[0, :][i][nonzeros_inds]
            data[1, :][i] = data[1, :][i][nonzeros_inds]
            data[2, :][i] = data[2, :][i][nonzeros_inds]

        evts = data_true[(data_true['edepR'] < R_upper) & (data_true['edepR'] > R_lower)].index
        data_same_RE = data[:, evts]
        data_true_same_RE = data_true.loc[evts]

        data_in_bins = []
        for i in range(data_same_RE.shape[1]):
            data_in_bin = []
            for j in range(len(bins)-1):
                value = np.where((data_same_RE[ind, i] >= bins[j]) & (data_same_RE[ind, i] <= bins[j+1]))[0].shape[0]
                data_in_bin.append(value)
            data_in_bins.append(data_in_bin)

        data_in_bins = np.array(data_in_bins)
        all_data_in_bins.append(data_in_bins)

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100997 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100997 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/99393 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/99307 [00:00<?, ?it/s]

In [ ]:
all_data_in_bins = np.array(all_data_in_bins).reshape(2, 2)

In [ ]:
if type_of_particle=='e+':
    edep_energy = energy+1
elif type_of_particle=='gamma':
    edep_energy = energy

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default = 'plotly_white'

colors = [['#0D2A63', '#FF6692'], ['red', 'darkviolet']]
names = [['e+ Ideal', 'e+ Real'], ['gamma Ideal', 'gamma Real']]

fig = make_subplots(rows=2, cols=1)
for i in range(1, 3):
    for j in range(1, 3):
        fig.add_trace(
            go.Bar(
                x=bins,
                y=all_data_in_bins[i-1][j-1].mean(axis=0),
                opacity=0.9,
                name=names[i-1][j-1],
                showlegend=True,
                marker=dict(
                    color=colors[i-1][j-1]
                ),
                error_y=dict(
                        type='data',
                        array=all_data_in_bins[i-1][j-1].std(axis=0),
                        visible=True,
                        width=8
                    ),
    #                 marker=dict(color=bar_color)
            ), row=j, col=1
        )

xaxis = dict(
    showline=True,
    ticks='outside',
    mirror=True,
    linecolor='black',
    showgrid=True,
    dtick=dtick,
    gridcolor='grey',
    gridwidth=0.25,
)

yaxis = dict(
    showline=True,
    ticks='outside',
    mirror=True,
    linecolor='black',
    showgrid=True,
    gridcolor='grey',
    gridwidth=0.25,
    range=y_range
)

fig.update_layout(
#     title='Energy = {} MeV, R = [{}-{}] m.'.format(
#         edep_energy, R_lower, R_upper),
    xaxis1_title=xaxis_title,
    xaxis2_title=xaxis_title,
    xaxis1 = xaxis,
    xaxis2 = xaxis,
    yaxis1 = yaxis,
    yaxis2 = yaxis,
    width=975,
    height=800,
#     barmode='overlay',
    font = dict(
        family="Times New Roman",
        size=16,
        color='black'
    ),
    legend = dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    ),
    bargap=0.1,
    bargroupgap=0
)

fig.show()
pio.write_image(
    fig, 'plots/plot_avg_signals_comparison_{}.pdf'.format(mode),
    width=975, height=800)